In [69]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [70]:
df = df.iloc[:300]

In [12]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\alexe\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alexe\.cache\huggingface\hub\models--sentence-transformers--multi-qa-mpnet-base-dot-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
q = df.iloc[0].answer_llm

In [16]:
v = embedding_model.encode(q)

In [17]:
v[0]

-0.4224466

In [71]:
records = df.to_dict(orient='records')

In [72]:
from tqdm.auto import tqdm

In [21]:
evaluations = []

for r in tqdm(records):
    u = embedding_model.encode(r['answer_llm'])
    v = embedding_model.encode(r['answer_orig'])
    evaluations.append(u.dot(v))

  0%|          | 0/300 [00:00<?, ?it/s]

In [23]:
evaluations = pd.Series(evaluations)
evaluations.describe()

count    300.000000
mean      27.495996
std        6.384743
min        4.547925
25%       24.307845
50%       28.336869
75%       31.674311
max       39.476021
dtype: float64

In [24]:
u.dot(u)

36.947308

In [36]:
import numpy as np

In [40]:
def normalize(v):
    return v / np.sqrt((v * v).sum())

In [42]:
vn.dot(vn)

1.0000001

In [43]:
evaluations = []

for r in tqdm(records):
    u = embedding_model.encode(r['answer_llm'])
    v = embedding_model.encode(r['answer_orig'])
    u = normalize(u)
    v = normalize(v)
    evaluations.append(u.dot(v))

  0%|          | 0/300 [00:00<?, ?it/s]

In [44]:
evaluations = pd.Series(evaluations)
evaluations.describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
dtype: float64

In [83]:
from rouge import Rouge
rouge_scorer = Rouge()

In [84]:
r = records[10]

In [85]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [86]:
scores['rouge-1']['f']

0.45454544954545456

In [87]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [88]:
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [89]:
rouge_avg

0.35490034990035496

In [90]:
evaluations = []

for r in tqdm(records):
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    
    evaluations.append({
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'rouge_avg': (rouge_1 + rouge_2 + rouge_l) / 3
    })

  0%|          | 0/300 [00:00<?, ?it/s]

In [93]:
pd.DataFrame(evaluations).rouge_2.mean()

0.20696501983423318